In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data =pd.read_csv("../input/housesalesprediction/kc_house_data.csv")

In [ ]:
data.head()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data.columns

In [ ]:
data.info() # Yeah, except for date every column is int / float type

In [ ]:
# Lets first check missing values
data.isna().sum() # No missing values, good!!!

In [ ]:
# Lets look at our target varibale

In [ ]:
sns.distplot(data.price) # seems like there is some outlier and skewed towards right

In [ ]:
sns.boxplot(data.price) # Lets drop the value over 1.25

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.price.describe()

In [ ]:
(data.price > 1250000).sum()

In [ ]:
df0 = data.copy()

In [ ]:
df0 = df0[~(data.price > 1250000)]
df0 = df0.reset_index(drop=True)

In [ ]:
df0.shape

In [ ]:
# Lets again look at distribution plot
sns.distplot(df0.price) # Close to normal distribution

In [ ]:
# Drpping id column
df0.drop('id', axis =1, inplace = True)

In [ ]:
# Lets solve this date column, it is giving me headache
df0[['date']]

In [ ]:
# Going through the data, date is in yyyymmdd format

In [ ]:
date_clm1 = pd.Series(v[0:8] for v in df0['date']) 

In [ ]:
df0['Date_new'] = date_clm1

In [ ]:
df1 = df0.copy()

In [ ]:
df1.isna().sum()

In [ ]:
df1.drop('date', axis =1 , inplace = True)

In [ ]:
# Convert to date object
df1['Date'] = pd.to_datetime(df1['Date_new'])

In [ ]:
df1['year'] = pd.DatetimeIndex(df1['Date']).year
df1['month'] = pd.DatetimeIndex(df1['Date']).month

In [ ]:
# Lets remove Date and Date_new column
df1.drop('Date_new', axis =1 , inplace = True)
df1.drop('Date', axis =1 , inplace = True)

In [ ]:
df1.head()

In [ ]:
df1.year.unique()

In [ ]:
df1.month.unique()

In [ ]:
df1.isna().sum()

In [ ]:
df2 = df1.copy()

In [ ]:
sns.boxplot(data = df2, x = 'year', y = 'price') # Lol, no difference, lets drop it or use to ctreate an ew feature

In [ ]:
sns.boxplot(data = df2, x = 'month', y = 'price') # Lol, again, not much difference, lets drop it

In [ ]:
df2.drop('month', axis =1, inplace = True)

In [ ]:
df2.columns

In [ ]:
for col in df2.columns:
    print(col, ":" ,len(df2[col].unique()))

In [ ]:
sns.boxplot(data = df2, x = 'waterfront', y = 'price') # Wow, there is clearly a difference

In [ ]:
sns.boxplot(data = df2, x = 'view', y = 'price') # Wow, there is clearly a difference

In [ ]:
sns.boxplot(data = df2, x = 'floors', y = 'price') # We can remove or one hot encode it

In [ ]:
sns.boxplot(data = df2, x = 'condition', y = 'price') # Not a significant diff, lets take a call at last

In [ ]:
corrmat = df2.corr()
corrmat

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))
sns.heatmap(corrmat.abs(), annot = True)

In [ ]:
df3 = df2.copy()

In [ ]:
df3 = df2.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = df3.drop('price', axis = 1)
y = df3['price']

In [ ]:
scaler = StandardScaler()

In [ ]:
X_s = scaler.fit_transform(X)
X_s

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X1_train, X1_test, y_train, y_test = train_test_split(X_s,y, test_size =0.3)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
model = LinearRegression()
model.fit(X1_train,y_train)

In [ ]:
y_predict = model.predict(X1_test)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
score = r2_score(y_test,y_predict)
score

In [ ]:
import xgboost as xgb

In [ ]:
xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

In [ ]:
xgb.fit(X1_train,y_train)

In [ ]:
xgb.fit(X1_train,y_train)

In [ ]:
predictions = xgb.predict(X1_test)
score1 = r2_score(y_test,predictions)

In [ ]:
score1

In [ ]:
# Lets do cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(xgb, X_s,y, cv =30).mean()

In [ ]:
# Thank you, long live Kaggle!